In [1]:
# imports
import os 
import pandas as pd 
import requests 

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [2]:
#Load API key from env variable 
api_key = os.environ["FOURSQUARE_API_KEY"]

In [3]:
def get_poi_fs(latitude, longitude, radius, api_key, categories):    
    url = f'https://api.foursquare.com/v3/places/search?ll={latitude},{longitude}&radius={radius}'

    # Creating dictionary for headers
    headers = {"Accept": "application/json"}
    headers['Authorization'] = api_key
    reponse = requests.get(url, headers=headers)
    reponse_data = reponse.json()
    
    return reponse_data['results']

In [4]:
# Testing result to ensure api_key is a variable containing Foursquare API key
res = get_poi_fs(41.425364,2.185207,1000, api_key, categories=['restaurants','bars','pubs'])

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [5]:
# Parsing above response to get POI
## putting above response in a readable format
res[0]

{'fsq_id': '4e18b61d18a8166f738f1ed0',
 'categories': [{'id': 13026,
   'name': 'BBQ Joint',
   'short_name': 'BBQ',
   'plural_name': 'BBQ Joints',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/bbqalt_',
    'suffix': '.png'}},
  {'id': 13338,
   'name': 'Seafood Restaurant',
   'short_name': 'Seafood',
   'plural_name': 'Seafood Restaurants',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/seafood_',
    'suffix': '.png'}},
  {'id': 13345,
   'name': 'Spanish Restaurant',
   'short_name': 'Spanish',
   'plural_name': 'Spanish Restaurants',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/spanish_',
    'suffix': '.png'}}],
 'chains': [],
 'closed_bucket': 'LikelyOpen',
 'distance': 215,
 'geocodes': {'main': {'latitude': 41.427162, 'longitude': 2.186739},
  'roof': {'latitude': 41.427162, 'longitude': 2.186739}},
 'link': '/v3/places/4e18b61d18a8166f738f1ed0',
 'location': {'address': 'Calle Torroella de Montgrí 7',
  'admin_r

Put your parsed results into a DataFrame

In [6]:
bike_stations_data1 = pd.read_csv('../data/bike_stations_barcelona.csv', index_col=0)

In [7]:
# Creating a DataFrame for Foursquare
foursquare_df = pd.DataFrame()

In [8]:
## Create empty dataframes
poi_fs = []
new_df_1 = []

In [9]:
## Looping through bike_stations_data1 Dataframe to extract lat, long, and id of each row 
for index, row in bike_stations_data1.iterrows():
    poi_fs = get_poi_fs(row['latitude'], row['longitude'], 1000, api_key, categories=None)

    # put response into dataframe called "new_df_1"
    new_df_1 = pd.json_normalize(poi_fs, record_path=['categories'],meta=['fsq_id', 'name', 'distance', ['location', 'address']], meta_prefix='poi_', errors='ignore')

    # Adding column with the corresponding bike station id
    new_df_1['station_id'] = row['id']

    # Adding new_df_1 dataframe to yelp dataframe 
    foursquare_df = pd.concat([foursquare_df, new_df_1])

In [10]:
# Dropping unnecessary columns/ empty columns from foursquare dataframe
columns_to_drop_foursquare = ['short_name', 'plural_name', 'icon.prefix', 'icon.suffix']

foursquare_df = foursquare_df.drop(columns=columns_to_drop_foursquare)

# Renaming columns
foursquare_df = foursquare_df.rename(columns={"id": "category_id", "name": "category_name", 'poi_location.address': 'poi_address'})

In [11]:
# Reordering the columns above
foursquare_df = foursquare_df[['poi_fsq_id', 'poi_name', 'category_id', 'category_name', 'poi_distance', 'poi_address', 'station_id']]

In [12]:
## Printing shape and info of Dataframe after removing NAN/empty columns, and reording columns
print(foursquare_df.shape, '\n')
print(foursquare_df.info())

(8042, 7) 

<class 'pandas.core.frame.DataFrame'>
Index: 8042 entries, 0 to 16
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   poi_fsq_id     8042 non-null   object
 1   poi_name       8042 non-null   object
 2   category_id    8042 non-null   int64 
 3   category_name  8042 non-null   object
 4   poi_distance   8042 non-null   object
 5   poi_address    7693 non-null   object
 6   station_id     8042 non-null   object
dtypes: int64(1), object(6)
memory usage: 502.6+ KB
None


In [13]:
foursquare_df.head()

,poi_fsq_id,poi_name,category_id,category_name,poi_distance,poi_address,station_id
0,4e18b61d18a8166f738f1ed0,Mesón Pahi,13026,BBQ Joint,215,Calle Torroella de Montgrí 7,dedcd93fbe85671e73c79baf58bd7fed
1,4e18b61d18a8166f738f1ed0,Mesón Pahi,13338,Seafood Restaurant,215,Calle Torroella de Montgrí 7,dedcd93fbe85671e73c79baf58bd7fed
2,4e18b61d18a8166f738f1ed0,Mesón Pahi,13345,Spanish Restaurant,215,Calle Torroella de Montgrí 7,dedcd93fbe85671e73c79baf58bd7fed
3,590896ec5d891b372346f974,Cyclic Beer Farm,13029,Brewery,344,"Carrer de Coll, 37",dedcd93fbe85671e73c79baf58bd7fed
4,590896ec5d891b372346f974,Cyclic Beer Farm,13065,Restaurant,344,"Carrer de Coll, 37",dedcd93fbe85671e73c79baf58bd7fed


In [14]:
## Save above data to use within another notebook 
foursquare_df.to_csv('../data/bike_stations_barcelona_poi_fsq.csv')

# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [15]:
import requests
import os
import pandas as pd 

In [16]:
## Loading environment variable for yelp using os.environment 
API_KEY = os.environ.get("yelp_api_key")

In [17]:
## Sending http request to Yelp API 
url = "https://api.yelp.com/v3/businesses/search?latitude=41.3874&longitude=2.1686&radius=1000&categories=bars%2C%20restaurant%2C%20pubs&sort_by=best_match&limit=20"

headers = {
    "accept": "application/json",
    "Authorization": "Bearer " + API_KEY
}

response = requests.get(url, headers=headers)
response_data = response.json()
print(response_data)

{'businesses': [{'id': 'Cn1geHmwrvKu1qQISgQKgA', 'alias': 'sensi-barcelona', 'name': 'Sensi', 'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/ndqinTlySy6slhGGeykyww/o.jpg', 'is_closed': False, 'url': 'https://www.yelp.com/biz/sensi-barcelona?adjust_creative=QyGIBXO3bewZz1kNqGAwCA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=QyGIBXO3bewZz1kNqGAwCA', 'review_count': 402, 'categories': [{'alias': 'tapasmallplates', 'title': 'Tapas/Small Plates'}, {'alias': 'tapas', 'title': 'Tapas Bars'}, {'alias': 'cocktailbars', 'title': 'Cocktail Bars'}], 'rating': 4.5, 'coordinates': {'latitude': 41.38035, 'longitude': 2.17997}, 'transactions': [], 'price': '€€', 'location': {'address1': 'Carrer Ample, 26', 'address2': '', 'address3': '', 'city': 'Barcelona', 'zip_code': '08002', 'country': 'ES', 'state': 'B', 'display_address': ['Carrer Ample, 26', '08002 Barcelona', 'Spain']}, 'phone': '+34932956588', 'display_phone': '+34 932 95 65 88', 'distance': 1237.3374168059947}

In [18]:
## Putting above code in more readable format
response.json()

{'businesses': [{'id': 'Cn1geHmwrvKu1qQISgQKgA',
   'alias': 'sensi-barcelona',
   'name': 'Sensi',
   'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/ndqinTlySy6slhGGeykyww/o.jpg',
   'is_closed': False,
   'url': 'https://www.yelp.com/biz/sensi-barcelona?adjust_creative=QyGIBXO3bewZz1kNqGAwCA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=QyGIBXO3bewZz1kNqGAwCA',
   'review_count': 402,
   'categories': [{'alias': 'tapasmallplates', 'title': 'Tapas/Small Plates'},
    {'alias': 'tapas', 'title': 'Tapas Bars'},
    {'alias': 'cocktailbars', 'title': 'Cocktail Bars'}],
   'rating': 4.5,
   'coordinates': {'latitude': 41.38035, 'longitude': 2.17997},
   'transactions': [],
   'price': '€€',
   'location': {'address1': 'Carrer Ample, 26',
    'address2': '',
    'address3': '',
    'city': 'Barcelona',
    'zip_code': '08002',
    'country': 'ES',
    'state': 'B',
    'display_address': ['Carrer Ample, 26', '08002 Barcelona', 'Spain']},
   'phone': '+3493295

In [19]:
# Retreiving info about first business within the list
print(response_data['businesses'])

[{'id': 'Cn1geHmwrvKu1qQISgQKgA', 'alias': 'sensi-barcelona', 'name': 'Sensi', 'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/ndqinTlySy6slhGGeykyww/o.jpg', 'is_closed': False, 'url': 'https://www.yelp.com/biz/sensi-barcelona?adjust_creative=QyGIBXO3bewZz1kNqGAwCA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=QyGIBXO3bewZz1kNqGAwCA', 'review_count': 402, 'categories': [{'alias': 'tapasmallplates', 'title': 'Tapas/Small Plates'}, {'alias': 'tapas', 'title': 'Tapas Bars'}, {'alias': 'cocktailbars', 'title': 'Cocktail Bars'}], 'rating': 4.5, 'coordinates': {'latitude': 41.38035, 'longitude': 2.17997}, 'transactions': [], 'price': '€€', 'location': {'address1': 'Carrer Ample, 26', 'address2': '', 'address3': '', 'city': 'Barcelona', 'zip_code': '08002', 'country': 'ES', 'state': 'B', 'display_address': ['Carrer Ample, 26', '08002 Barcelona', 'Spain']}, 'phone': '+34932956588', 'display_phone': '+34 932 95 65 88', 'distance': 1237.3374168059947}, {'id': 'Fb7Ni

In [20]:
## Retrieving the first business within the list 
response_data['businesses'][0]

{'id': 'Cn1geHmwrvKu1qQISgQKgA',
 'alias': 'sensi-barcelona',
 'name': 'Sensi',
 'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/ndqinTlySy6slhGGeykyww/o.jpg',
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/sensi-barcelona?adjust_creative=QyGIBXO3bewZz1kNqGAwCA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=QyGIBXO3bewZz1kNqGAwCA',
 'review_count': 402,
 'categories': [{'alias': 'tapasmallplates', 'title': 'Tapas/Small Plates'},
  {'alias': 'tapas', 'title': 'Tapas Bars'},
  {'alias': 'cocktailbars', 'title': 'Cocktail Bars'}],
 'rating': 4.5,
 'coordinates': {'latitude': 41.38035, 'longitude': 2.17997},
 'transactions': [],
 'price': '€€',
 'location': {'address1': 'Carrer Ample, 26',
  'address2': '',
  'address3': '',
  'city': 'Barcelona',
  'zip_code': '08002',
  'country': 'ES',
  'state': 'B',
  'display_address': ['Carrer Ample, 26', '08002 Barcelona', 'Spain']},
 'phone': '+34932956588',
 'display_phone': '+34 932 95 65 88',
 'distance': 12

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [21]:
# Creating list with desired categories from Yelp
dining_cat_yelp = ['breweries', 'restaurants', 'pubs']

In [22]:
# Creating function and making request to yelp for each bike station
def get_poi_yelp(latitude, longitude, radius, API_KEY, categories):
    url = f'https://api.yelp.com/v3/businesses/search?latitude={latitude}&longitude={longitude}&radius={radius}&categories={categories}&sort_by=best_match'
    
    # Create dictionary for headers
    headers = {'Accept': 'application/json'}
    headers['Authorization'] = 'Bearer %s' % API_KEY
    response = requests.get(url, headers=headers)
    data = response.json()  
    data['station_id']= 'dedcd93fbe85671e73c79baf58bd7fed' ##this is the station id for "C/ CARDENAL TEDESCHINI"
    return data['businesses']

In [23]:
## Testing results
res_2 = get_poi_yelp(41.4253643, 2.1852073, 1000, API_KEY, categories=','.join(dining_cat_yelp))

In [24]:
## Exploring POI from testing call by printing variable
print(res_2)

[{'id': 'J1vHLZvSRfkNpy2x8OuVaQ', 'alias': 'torre-rosa-barcelona-2', 'name': 'Torre Rosa', 'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/HPgkN8pNn5Dz3J-2bxmK0w/o.jpg', 'is_closed': False, 'url': 'https://www.yelp.com/biz/torre-rosa-barcelona-2?adjust_creative=QyGIBXO3bewZz1kNqGAwCA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=QyGIBXO3bewZz1kNqGAwCA', 'review_count': 21, 'categories': [{'alias': 'cocktailbars', 'title': 'Cocktail Bars'}, {'alias': 'pubs', 'title': 'Pubs'}], 'rating': 4.0, 'coordinates': {'latitude': 41.422683, 'longitude': 2.1803748}, 'transactions': [], 'price': '€€', 'location': {'address1': 'Carrer de Francesc Tàrrega, 22', 'address2': '', 'address3': None, 'city': 'Barcelona', 'zip_code': '08027', 'country': 'ES', 'state': 'B', 'display_address': ['Carrer de Francesc Tàrrega, 22', '08027 Barcelona', 'Spain']}, 'phone': '+34933408854', 'display_phone': '+34 933 40 88 54', 'distance': 501.23627447031674}, {'id': 'oD9g3jw6UoebuiI3GtSGww

In [25]:
## Printing above in readable format 
res_2

[{'id': 'J1vHLZvSRfkNpy2x8OuVaQ',
  'alias': 'torre-rosa-barcelona-2',
  'name': 'Torre Rosa',
  'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/HPgkN8pNn5Dz3J-2bxmK0w/o.jpg',
  'is_closed': False,
  'url': 'https://www.yelp.com/biz/torre-rosa-barcelona-2?adjust_creative=QyGIBXO3bewZz1kNqGAwCA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=QyGIBXO3bewZz1kNqGAwCA',
  'review_count': 21,
  'categories': [{'alias': 'cocktailbars', 'title': 'Cocktail Bars'},
   {'alias': 'pubs', 'title': 'Pubs'}],
  'rating': 4.0,
  'coordinates': {'latitude': 41.422683, 'longitude': 2.1803748},
  'transactions': [],
  'price': '€€',
  'location': {'address1': 'Carrer de Francesc Tàrrega, 22',
   'address2': '',
   'address3': None,
   'city': 'Barcelona',
   'zip_code': '08027',
   'country': 'ES',
   'state': 'B',
   'display_address': ['Carrer de Francesc Tàrrega, 22',
    '08027 Barcelona',
    'Spain']},
  'phone': '+34933408854',
  'display_phone': '+34 933 40 88 54',
  '

In [26]:
response_data['businesses'][0]

{'id': 'Cn1geHmwrvKu1qQISgQKgA',
 'alias': 'sensi-barcelona',
 'name': 'Sensi',
 'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/ndqinTlySy6slhGGeykyww/o.jpg',
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/sensi-barcelona?adjust_creative=QyGIBXO3bewZz1kNqGAwCA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=QyGIBXO3bewZz1kNqGAwCA',
 'review_count': 402,
 'categories': [{'alias': 'tapasmallplates', 'title': 'Tapas/Small Plates'},
  {'alias': 'tapas', 'title': 'Tapas Bars'},
  {'alias': 'cocktailbars', 'title': 'Cocktail Bars'}],
 'rating': 4.5,
 'coordinates': {'latitude': 41.38035, 'longitude': 2.17997},
 'transactions': [],
 'price': '€€',
 'location': {'address1': 'Carrer Ample, 26',
  'address2': '',
  'address3': '',
  'city': 'Barcelona',
  'zip_code': '08002',
  'country': 'ES',
  'state': 'B',
  'display_address': ['Carrer Ample, 26', '08002 Barcelona', 'Spain']},
 'phone': '+34932956588',
 'display_phone': '+34 932 95 65 88',
 'distance': 12

In [27]:
## Loading the dataframe of bike stations that contain lat/long 
bike_stations_data2 = pd.read_csv('../data/bike_stations_barcelona.csv', index_col=0)

In [28]:
## Creating empty dataframe for yelp 
yelp_df = pd.DataFrame()

In [29]:
print(yelp_df.shape, '\n')
print(yelp_df.info())

(0, 0) 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Empty DataFrame
None


In [30]:
# Set the maximum number of API calls
max_api_calls = 40

# Counter for the number of API calls made
api_calls_count = 0

# Loop through the bike_stations_data2 dataframe to extract the lat/long/id of each row and make a new request to Yelp API
for index, row in bike_stations_data2.iterrows():
    # Make the http request using row attributes
    poi_yelp = get_poi_yelp(row['latitude'], row['longitude'], 1000, API_KEY, categories=','.join(dining_cat_yelp))
    
    # Check if the limit of API calls has been maximized 
    if api_calls_count >= max_api_calls:
        break

    # Turn the response into a dataframe
    new_df = pd.json_normalize(poi_yelp)
    # Add a column with the corresponding bike station id
    new_df['station_id'] = row['id'] 

    # Add the new dataframe to yelp_df
    yelp_df = pd.concat([yelp_df, new_df]) 

    # Increment the API calls counter
    api_calls_count += 1

In [31]:
print(yelp_df.shape, '\n')
print(yelp_df.info())

(792, 25) 

<class 'pandas.core.frame.DataFrame'>
Index: 792 entries, 0 to 19
Data columns (total 25 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   id                        792 non-null    object 
 1   alias                     792 non-null    object 
 2   name                      792 non-null    object 
 3   image_url                 792 non-null    object 
 4   is_closed                 792 non-null    bool   
 5   url                       792 non-null    object 
 6   review_count              792 non-null    int64  
 7   categories                792 non-null    object 
 8   rating                    792 non-null    float64
 9   transactions              792 non-null    object 
 10  price                     641 non-null    object 
 11  phone                     792 non-null    object 
 12  display_phone             792 non-null    object 
 13  distance                  792 non-null    float64
 14  coor

In [32]:
# Drop unnecessary columns from dataframe
columns_to_drop_yelp = [
    'alias', 'image_url', 'is_closed', 'url', 'transactions', 'phone',
    'display_phone', 'coordinates.latitude',
    'coordinates.longitude', 'location.address1', 'location.address2',
    'location.address3', 'location.city', 'location.zip_code',
    'location.country', 'location.state','categories']

yelp_df = yelp_df.drop(columns=columns_to_drop_yelp)

# Rename columns
yelp_df = yelp_df.rename(columns={"id": "poi_id", "location.display_address": "full_address"})

In [33]:
print(yelp_df.shape, '\n')
print(yelp_df.info())

(792, 8) 

<class 'pandas.core.frame.DataFrame'>
Index: 792 entries, 0 to 19
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   poi_id        792 non-null    object 
 1   name          792 non-null    object 
 2   review_count  792 non-null    int64  
 3   rating        792 non-null    float64
 4   price         641 non-null    object 
 5   distance      792 non-null    float64
 6   full_address  792 non-null    object 
 7   station_id    792 non-null    object 
dtypes: float64(2), int64(1), object(5)
memory usage: 55.7+ KB
None


In [34]:
## Printing dataframe to return first n rows 
print(yelp_df.head())

                   poi_id                          name  review_count  rating  \
0  J1vHLZvSRfkNpy2x8OuVaQ                    Torre Rosa            21     4.0   
1  oD9g3jw6UoebuiI3GtSGww                  Tasca I Vins            10     4.0   
2  e6bRub-8J0j5PDm55UNASA                     Toca Teca             3     4.5   
3  iRrsa0bQbTtHzBjsHIprqQ             Beering Barcelona            10     4.5   
4  UdRmo2dstOYd29Ihs5vI8w  La Vermuteria de Sant Andreu             4     4.0   

  price    distance                                       full_address  \
0    €€  501.236274  [Carrer de Francesc Tàrrega, 22, 08027 Barcelo...   
1     €  191.557179  [Calle Cardenal Tedeschini, 32, 08027 Barcelon...   
2     €  395.734907    [Carrer Garcilaso, 172, 08027 Barcelona, Spain]   
3     €  828.625543  [Carrer de las Navas de Tolosa, 355, 08041 Bar...   
4    €€  508.573281  [Rambla de Fabra I Puig, 51, 08030 Barcelona, ...   

                         station_id  
0  dedcd93fbe85671e73c79baf58b

In [35]:
## Dropping price and review count columns from dataframe
columns_to_drop_yelp = ['review_count', 'price']
yelp_df = yelp_df.drop(columns=columns_to_drop_yelp)

In [36]:
# Displaying dataframe with columns that have been removed 
yelp_df.head()

,poi_id,name,rating,distance,full_address,station_id
0,J1vHLZvSRfkNpy2x8OuVaQ,Torre Rosa,4.0,501.236274,"[Carrer de Francesc Tàrrega, 22, 08027 Barcelo...",dedcd93fbe85671e73c79baf58bd7fed
1,oD9g3jw6UoebuiI3GtSGww,Tasca I Vins,4.0,191.557179,"[Calle Cardenal Tedeschini, 32, 08027 Barcelon...",dedcd93fbe85671e73c79baf58bd7fed
2,e6bRub-8J0j5PDm55UNASA,Toca Teca,4.5,395.734907,"[Carrer Garcilaso, 172, 08027 Barcelona, Spain]",dedcd93fbe85671e73c79baf58bd7fed
3,iRrsa0bQbTtHzBjsHIprqQ,Beering Barcelona,4.5,828.625543,"[Carrer de las Navas de Tolosa, 355, 08041 Bar...",dedcd93fbe85671e73c79baf58bd7fed
4,UdRmo2dstOYd29Ihs5vI8w,La Vermuteria de Sant Andreu,4.0,508.573281,"[Rambla de Fabra I Puig, 51, 08030 Barcelona, ...",dedcd93fbe85671e73c79baf58bd7fed


In [37]:
# Assuming DataFrame is named df
businesses_col = yelp_df['distance']

# Display the 'distance' column of above output 
print(businesses_col)

0     501.236274
1     191.557179
2     395.734907
3     828.625543
4     508.573281
         ...    
15    502.544785
16    694.280017
17    559.148884
18    588.751811
19    671.128365
Name: distance, Length: 792, dtype: float64


In [38]:
yelp_df.head()['distance']

0    501.236274
1    191.557179
2    395.734907
3    828.625543
4    508.573281
Name: distance, dtype: float64

In [39]:
yelp_df.head()

,poi_id,name,rating,distance,full_address,station_id
0,J1vHLZvSRfkNpy2x8OuVaQ,Torre Rosa,4.0,501.236274,"[Carrer de Francesc Tàrrega, 22, 08027 Barcelo...",dedcd93fbe85671e73c79baf58bd7fed
1,oD9g3jw6UoebuiI3GtSGww,Tasca I Vins,4.0,191.557179,"[Calle Cardenal Tedeschini, 32, 08027 Barcelon...",dedcd93fbe85671e73c79baf58bd7fed
2,e6bRub-8J0j5PDm55UNASA,Toca Teca,4.5,395.734907,"[Carrer Garcilaso, 172, 08027 Barcelona, Spain]",dedcd93fbe85671e73c79baf58bd7fed
3,iRrsa0bQbTtHzBjsHIprqQ,Beering Barcelona,4.5,828.625543,"[Carrer de las Navas de Tolosa, 355, 08041 Bar...",dedcd93fbe85671e73c79baf58bd7fed
4,UdRmo2dstOYd29Ihs5vI8w,La Vermuteria de Sant Andreu,4.0,508.573281,"[Rambla de Fabra I Puig, 51, 08030 Barcelona, ...",dedcd93fbe85671e73c79baf58bd7fed


Put your parsed results into a DataFrame

In [40]:
## Saving this data to use within another notebook 
yelp_df.to_csv('../data/bike_stations_barcelona_poi_yelp.csv')

In [41]:

# Load the CSV file into a DataFrame
df = pd.read_csv('../data/bike_stations_barcelona_poi_yelp.csv')

# Comparing Results

In [42]:
## Printing Foursquare API Results 
print('foursquare results')
foursquare_df.head()

foursquare results


,poi_fsq_id,poi_name,category_id,category_name,poi_distance,poi_address,station_id
0,4e18b61d18a8166f738f1ed0,Mesón Pahi,13026,BBQ Joint,215,Calle Torroella de Montgrí 7,dedcd93fbe85671e73c79baf58bd7fed
1,4e18b61d18a8166f738f1ed0,Mesón Pahi,13338,Seafood Restaurant,215,Calle Torroella de Montgrí 7,dedcd93fbe85671e73c79baf58bd7fed
2,4e18b61d18a8166f738f1ed0,Mesón Pahi,13345,Spanish Restaurant,215,Calle Torroella de Montgrí 7,dedcd93fbe85671e73c79baf58bd7fed
3,590896ec5d891b372346f974,Cyclic Beer Farm,13029,Brewery,344,"Carrer de Coll, 37",dedcd93fbe85671e73c79baf58bd7fed
4,590896ec5d891b372346f974,Cyclic Beer Farm,13065,Restaurant,344,"Carrer de Coll, 37",dedcd93fbe85671e73c79baf58bd7fed


In [43]:
## Printing Yelp API results
print('Yelp results')
yelp_df.head()

Yelp results


,poi_id,name,rating,distance,full_address,station_id
0,J1vHLZvSRfkNpy2x8OuVaQ,Torre Rosa,4.0,501.236274,"[Carrer de Francesc Tàrrega, 22, 08027 Barcelo...",dedcd93fbe85671e73c79baf58bd7fed
1,oD9g3jw6UoebuiI3GtSGww,Tasca I Vins,4.0,191.557179,"[Calle Cardenal Tedeschini, 32, 08027 Barcelon...",dedcd93fbe85671e73c79baf58bd7fed
2,e6bRub-8J0j5PDm55UNASA,Toca Teca,4.5,395.734907,"[Carrer Garcilaso, 172, 08027 Barcelona, Spain]",dedcd93fbe85671e73c79baf58bd7fed
3,iRrsa0bQbTtHzBjsHIprqQ,Beering Barcelona,4.5,828.625543,"[Carrer de las Navas de Tolosa, 355, 08041 Bar...",dedcd93fbe85671e73c79baf58bd7fed
4,UdRmo2dstOYd29Ihs5vI8w,La Vermuteria de Sant Andreu,4.0,508.573281,"[Rambla de Fabra I Puig, 51, 08030 Barcelona, ...",dedcd93fbe85671e73c79baf58bd7fed


## Which API provided you with more complete data? Provide an explanation ##
When comparing the two tables above, I found that the foursquare API provides more detailed information (poi_fsq_id, poi_name, category_id, etc.) compared to the Yelp API table. In the Yelp API table, there are fewer columns and details. One comparable column between the two tables is station_id. Overall, I find that the Foursquare API provides more information compared to the Yelp API. 

Get the top 10 restaurants according to their rating

In [44]:
# Making copy of the yelp dataframe with poi only
yelp_unique_poi = yelp_df[['poi_id', 'name', 'rating', 'distance', 'full_address', 'station_id']].copy()

# Convert the 'full_address' column to strings
yelp_unique_poi['full_address'] = yelp_unique_poi['full_address'].astype(str)

# Drop any duplicates based on all columns
yelp_unique_poi.drop_duplicates(inplace=True, ignore_index=True)

# Display the result DataFrame
print(yelp_unique_poi.head())

                   poi_id                          name  rating    distance  \
0  J1vHLZvSRfkNpy2x8OuVaQ                    Torre Rosa     4.0  501.236274   
1  oD9g3jw6UoebuiI3GtSGww                  Tasca I Vins     4.0  191.557179   
2  e6bRub-8J0j5PDm55UNASA                     Toca Teca     4.5  395.734907   
3  iRrsa0bQbTtHzBjsHIprqQ             Beering Barcelona     4.5  828.625543   
4  UdRmo2dstOYd29Ihs5vI8w  La Vermuteria de Sant Andreu     4.0  508.573281   

                                        full_address  \
0  ['Carrer de Francesc Tàrrega, 22', '08027 Barc...   
1  ['Calle Cardenal Tedeschini, 32', '08027 Barce...   
2  ['Carrer Garcilaso, 172', '08027 Barcelona', '...   
3  ['Carrer de las Navas de Tolosa, 355', '08041 ...   
4  ['Rambla de Fabra I Puig, 51', '08030 Barcelon...   

                         station_id  
0  dedcd93fbe85671e73c79baf58bd7fed  
1  dedcd93fbe85671e73c79baf58bd7fed  
2  dedcd93fbe85671e73c79baf58bd7fed  
3  dedcd93fbe85671e73c79baf58bd7fed 

In [45]:
## Filtering above results by rating, name, and distance
yelp_unique_poi.sort_values(by=['name', 'rating', 'distance'], ascending=False).head(10)

,poi_id,name,rating,distance,full_address,station_id
629,rOiwFoXOZTp1yxVLCyC7Dg,Öuzo,5.0,360.455378,"['Rambla de Fabra i Puig, 66', '08030 Barcelon...",a08c92d249cec30792553ab20536cf99
35,rOiwFoXOZTp1yxVLCyC7Dg,Öuzo,5.0,316.560302,"['Rambla de Fabra i Puig, 66', '08030 Barcelon...",5c10c2dba1c5a8e8d9dcbb251fd8eab7
439,rOiwFoXOZTp1yxVLCyC7Dg,Öuzo,5.0,137.129150,"['Rambla de Fabra i Puig, 66', '08030 Barcelon...",d838e012f31b57bf9dc347842df5d1d6
369,zeVaxJ3XnougQ1sTQemfJw,las Cuatro Efes,5.0,1271.147945,"['Calle Segimon 10', '08032 Barcelona', 'Spain']",c1e818c978c673efb4fb8da5e7e51791
254,BCVlmrwt7tlY9s2S0ZhDgw,la masia,3.0,520.446950,"['alts forns, 61 calle', '08032 Barcelona', 'S...",4adedd174616ff28ded24a9ad859660f
92,I-c4V9IEE9KZ4ZmhoA9rDQ,Zarautz,4.5,823.153641,"[""Carrer de l'Elisi, 13"", '08014 Barcelona', '...",e964297fbcd2fc96bb6632e5a530d0df
713,I-c4V9IEE9KZ4ZmhoA9rDQ,Zarautz,4.5,386.581873,"[""Carrer de l'Elisi, 13"", '08014 Barcelona', '...",2859e381ef181038d89fdc04ef067165
532,I-c4V9IEE9KZ4ZmhoA9rDQ,Zarautz,4.5,282.374477,"[""Carrer de l'Elisi, 13"", '08014 Barcelona', '...",bd53dacde784bdd4a6fbea912d0fff75
232,Zx7QKQZzW4qdd1wlIL7Chg,Yon Gua,5.0,113.471528,"['Calle Maresme 197', '08020 Barcelona', 'Spain']",f28ea408c98c269ec16dd1a110ba77cc
203,7OYBlCoI6c9SnDyOwojPTQ,Xorus,4.5,1172.516306,"['Rambla de Poblenou, 105', '08005 Barcelona',...",de0e8ba6e1c44e2b7c452a639777e19a
